# AoC 2020 --- Day 20: Jurassic Jigsaw ---


Nous voici au 20ème jour de notre périple, franchement je ne sais plus si je vais réussir à atteindre la destination des vacances tant le parcours est semé d'obstacles.

Bref, je suis dans le train, et je me dirige vers le sud, et pour une fois tout semble bien se dérouler. 
Je sors donc de mon sac cette image que j'ai fini avec tant de mal à déchiffrer ... où pensent-ils avoir vu un monstre là dedans ???

Mouais bon comme d'hab, c'est pas simple ... me voilà avec des bouts d'images, sur fond transparent ... ah la la, ça va être pénible de les assembler pour obtenir l'image finale ...

Bon allez, je m'y mets, c'est pas comme si j'avais autre chose à faire de toute façon ...


## Les fragments d'image

Je me retrouve donc avec des fragments d'image à assembler. Chaque fragment est un **carré**, j'insiste, j'ai perdu un peu de temps à croire que c'étaient des rectangles...

Un fragment se présente de la sorte :

```
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###
```

Pour coller les fragments d'image entre eux il va falloir trouver un autre fragment dont le bord est identique.
De plus, les bords du puzzle ont pour caractéristique d'être uniques, on ne peut donc pas les retrouver sur les autres fragments.

En résumé :
- une image se colle à une autre si leurs bords sont identiques
- une image se situe au bord du puzzle si son bord est unique

### Manipulation des fragments

Pour commencer, je vais devoir être capable de tourner mes fragments d'images dans tous les sens.
Partons d'un exemple pour montrer ce que l'on souhaite obtenir.

A partir du fragment :

```
123
456
789
```
je dois être capable d'obtenir :
```
123   741   987   369   321   147   963   789 
456   852   654   258   654   258   852   456
789   963   321   147   987   369   741   123
```

In [1]:
fun String.rotations(): List<String> {
    val flip: String.() -> String = { lines().map { it.toList().reversed().joinToString("") }.joinToString("\n") }
    val rotate180: String.() -> String = { lines().reversed().joinToString("\n") }
    val rotate90: String.() -> String = { lines().mapIndexed { y, line ->
        line.toList().mapIndexed { x, _ ->
                lines()[x].toList()[y].toString()
            }.joinToString("")
        }
            .joinToString("\n")
            .flip()
    }

    val rotate270: String.() -> String = { rotate90().rotate180().flip() }

    return listOf(rotate180(), rotate270(), rotate90(), flip(), this, flip().rotate90(), flip().rotate180(), flip().rotate270())
}

// testons pour voir
"""
   123
   456
   789
""".trimIndent()
    .rotations()
    .joinToString("\n---\n")

789
456
123
---
369
258
147
---
741
852
963
---
321
654
987
---
123
456
789
---
963
852
741
---
987
654
321
---
147
258
369

Bon c'est cool, je suis capable de tourner mes fragments dans tous les sens !

Je me rends compte que je vais avoir besoin d'extraire les 4 bords d'un fragment afin de pouvoir les tester avec les bords des autres fragments.

Prenons
```
123
456
789
```

Je veux pouvoir demander gauche, droite, haut, bas et obtenir
```
147   369   123   789
```

C'est parti :

In [2]:
fun String.sideUp() = lines().first()

fun String.sideDown() = lines().last()

fun String.sideLeft() = lines().map {
            it.toList().first().toString()
        }.joinToString("")

fun String.sideRight() = lines().map { it.toList().last().toString() }.joinToString("")

// ça marche ?
"""
   123
   456
   789
""".trimIndent().let {
    "   ${it.sideLeft()}   ${it.sideRight()}   ${it.sideUp()}   ${it.sideDown()}"
}

   147   369   123   789

### Synthétisation d'un fragment

Je suis donc capable d'obtenir 
- tous les sens possibles pour un fragment
- pour chaque sens, ses 4 bords

Un fragment a
- 8 rotations possibles
- 1 seul sens d'affichage dans l'image finale (obviousment)
- un identifiant (x) pour pouvoir euh, comment dire, l'identifier

*(x) oui ça je l'avais pas dit avant mais chaque fragment m'est fourni accompagné d'un nombre permettant de l'identifier*

In [3]:
data class Tile(val id: Long, var display: String) {

    // alors là c'est bizarre, mais Jupyter veut pas ...
    // alors tu peux oublier ces qques lignes et considérer que j'ai écrit :
    // 
    // val rotations = display.rotations() // ça marche dans un vrai IDE, si si !
    
    var rotations = emptyList<String>()
            get() {
                if (field.isEmpty()) field = display.rotations()
                return field
            }
            private set
}

// ça c'est juste pour Jupyter, ... sinon il rale pour instancier l'objet hors de la cellule
val TileBuilder: (Long, String) -> Tile = { id, display -> Tile(id, display) }

## Le résolveur du puzzle

Maintenant que j'ai tout ce qu'il me faut pour manipuler mes fragments d'image, il va me falloir de quoi résoudre mon puzzle.
Bon alors il a quelle forme ce puzzle ? Ca c'est dans l'énoncé, c'est un carré. Ok.

Ben alors go, résolvons.

Alors admettons qu'on aie 9 fragments d'images, ça va nous faire un puzzle de 3x3.
On va essayer de placer les pièces les unes à côté des autres, en appliquant à chaque fois les contraintes qui s'appliquent : bord ou pièces adjacentes.

Je vais le remplir case par case :
```
[1]->[2]->[3]-+
              |
 +------------+
 |
[4]->[5]->[6]-+
              |
 +------------+
 |
[7]->[8]->[9]
```

Un fragment pourra avoir des contraintes sur ses 4 faces.

Il y a trois types de contraintes :
- le côté du fragment ne doit correspondre à aucun côté d'aucun des autres fragments
- le côté du fragment doit correspondre au côté du fragment à côté duquel on veut le placer
- le côté du fragment n'a pas de contrainte connue : il n'est ni au bord du puzzle, ni adjacent à un autre fragment.

In [4]:
class SquarePuzzleMaker(val tiles: List<Tile>) {
    
    // la taille d'un coté puzzle
    private val size = sqrt(tiles.size.toFloat()).toInt()

    // le puzzle à remplir, j'instancie tous ses emplacements avec la valeur null
    val puzzle: List<MutableList<Tile?>> = 
        (0 until size).map { 
            mutableListOf(*(0 until size).map { null }.toTypedArray()) 
        }
    
    // pas de contrainte : le coté est toujours valide
    private val noConstraint: (String, List<Tile>) -> Boolean = { _, _ -> true }
    
    // le coté ne doit correspondre à aucun des cotés des autres fragments
    private val noMatchConstraint: (String, List<Tile>) -> Boolean = { t, a -> a.none { it.run { rotations.any { it.sideRight() == t } } } }
    
    // le coté doit correspondre à celui du fragment à coté duquel on souhaite le placer
    private fun matchConstraint(expectedMatch: String): (String, List<Tile>) -> Boolean = { t, _ ->  t == expectedMatch }
    
    // récupération de la contrainte s'appliquant à gauche d'une case
    private fun leftConstraint(line: Int, column: Int) = 
        puzzle.get(line).getOrNull(column - 1)?.let { leftTile ->
            matchConstraint(leftTile.display.sideRight()) 
        } ?: if (column == 0) noMatchConstraint else noConstraint

    // récupération de la contrainte s'appliquant à droite d'une case
    private fun rightConstraint(line: Int, column: Int) = 
        puzzle.get(line).getOrNull(column + 1)?.let { rightTile -> 
            matchConstraint(rightTile.display.sideLeft()) 
        } ?: if (column == size - 1) noMatchConstraint else noConstraint

    // récupération de la contrainte s'appliquant en haut d'une case
    private fun upConstraint(line: Int, column: Int) = 
        puzzle.getOrNull(line - 1)?.get(column)?.let { upTile -> 
            matchConstraint(upTile.display.sideDown())
        } ?: if (line == 0) noMatchConstraint else noConstraint

    // récupération de la contrainte s'appliquant en bas d'une case
    private fun downConstraint(line: Int, column: Int) = 
        puzzle.getOrNull(line + 1)?.get(column)?.let { downTile -> 
            matchConstraint(downTile.display.sideUp())
        } ?: if (line == size - 1) noMatchConstraint else noConstraint
    
    fun fill(line: Int = 0, column: Int = 0, availableTiles: List<Tile> = tiles): Boolean {
            return if (availableTiles.isEmpty()) puzzle.none { it.any { it == null } }
            else {
                val leftConstraint = leftConstraint(line, column)
                val rightConstraint = rightConstraint(line, column)
                val upConstraint = upConstraint(line, column)
                val downConstraint = downConstraint(line, column)
                
                // pour chaque fragments j'essaye ceux qui valident les contraintes
                availableTiles.fold(false) { puzzleEndedForTile, tile ->
                    puzzleEndedForTile || 
                    // pour chaque rotation d'un fragment j'essaye celles qui valident les contraintes
                    tile.rotations.fold(puzzleEndedForTile) { puzzleEndedForRotation, rotation ->
                        puzzleEndedForRotation ||
                            // je soustrais des fragments disponibles celui que je vais essayer
                            availableTiles.minusElement(tile).let { newAvailableTiles ->
                                
                                // si la rotation valide ses 4 contraintes
                                // je fixe l'affichage du fragment sur cette rotation 
                                // et je remplis le puzzle
                                if (leftConstraint(rotation.sideLeft(), newAvailableTiles)
                                    && rightConstraint(rotation.sideRight(), newAvailableTiles)
                                    && upConstraint(rotation.sideUp(), newAvailableTiles)
                                    && downConstraint(rotation.sideDown(), newAvailableTiles)
                                ) {

                                    tile.display = rotation
                                    puzzle[line][column] = tile

                                    // puis je tente de remplir la case suivante du puzzle avec les fragments restants
                                    if (column == size - 1) fill(line + 1, 0, newAvailableTiles) 
                                    else fill(line, column + 1, newAvailableTiles)
                                } else {
                                    // si la rotation ne valide pas les 4 contraintes, je passe à la suivante
                                    puzzle[line][column] = null
                                    false
                                }
                        }
                    }
                }
            }
        }
}

// ça c'est juste pour Jupyter, ... sinon il rale pour instancier l'objet hors de la cellule
val PuzzleMaker: (List<Tile>) -> SquarePuzzleMaker = { l -> SquarePuzzleMaker(l) }

Le résolveur de puzzle est prêt, ya plus qu'à soumettre et croiser les doigts.

Voici les fragments :

In [5]:
val tiles = """Tile 1409:
##..#.#.#.
##........
#.#...##.#
#..#..#...
.......##.
##......##
..........
.........#
.#..##....
#.##...##.

Tile 2939:
......#.##
##.#......
...##...##
#.#.....##
#...#....#
.#..#....#
#.....##.#
..##.#...#
..#.#.#..#
#######..#

Tile 3347:
...##..#.#
.#......#.
.#........
#.....#...
#.....##..
##.......#
.....#....
......###.
#...#..##.
########.#

Tile 1297:
#..#.##.##
#..###...#
#.##......
...#.#...#
#.#......#
....#....#
.#..#.....
......##..
#.........
...#.##.##

Tile 3203:
####.#.#..
#.#.#.##..
#......###
#....###.#
.......#.#
#.........
#..#..##..
..##...#.#
#.....##..
#.##.#...#

Tile 1283:
####..#...
#.......##
#....#..#.
..##.....#
.#...#####
###...#...
..##....#.
.#.......#
.##.#.....
#.###..###

Tile 1879:
######.#..
..#.#....#
..#..##...
.#...#.#..
....#....#
....#.#.##
##.......#
#...#..#.#
..#.##....
#..####.#.

Tile 2293:
#.##.###.#
..#.....##
#...#.....
..##......
.#...#.#.#
#........#
.##...###.
###.#....#
...#......
.#..######

Tile 3079:
#.###.....
......#...
..##......
..#...#...
.#.#......
#....#...#
........##
..#..#...#
#..#......
#.#.#.###.

Tile 1069:
.#.##.....
...##...#.
###.#..##.
.#....#.##
......#.#.
#.#..#.##.
...#......
#..##...#.
##.##.....
#.#.##.#..

Tile 1229:
#.#...#..#
.........#
....#..##.
#.#...#..#
...###.#.#
##.##.....
...##....#
..#..#.###
..#......#
.##..#.#.#

Tile 3631:
###.....##
#.#.......
#.#.#..#.#
....#...##
#.###.#.#.
.....##.##
...#..###.
#..#...#.#
..#..##..#
.##.#.#..#

Tile 1747:
..##.....#
.#.....#.#
..........
..#.#.#..#
#...#.....
..##.#.#..
#....#..##
...#.....#
#.....##.#
..#.#####.

Tile 2531:
.#...##...
#....####.
##...##..#
#..#...#..
##....#...
.#....#.##
.........#
.#......#.
...#...#.#
##....#...

Tile 2203:
##..#...##
##..#.#..#
....##..#.
###.###...
.......#.#
#.....##.#
#.#....#..
.......#.#
...#.#....
##.#.####.

Tile 2777:
...#.##...
...#......
.#....##..
....#..#.#
#.....#..#
......##..
#....##.##
......#..#
#..##.##..
...#.#.###

Tile 3323:
#...##.#.#
..#.......
#......#..
.#..##...#
......##..
#....#..##
.......#.#
....#...#.
#.....#...
##..#.#..#

Tile 3319:
#....#.##.
##.....#.#
####......
..#.##..##
##....##.#
.......#.#
..######.#
#.#......#
#..#......
#..#.#..##

Tile 3313:
...#.##.##
........##
##..#...##
##.....#.#
##..#....#
.......#..
##.......#
#.#.......
.........#
#.#..###.#

Tile 2657:
.###..#..#
..#......#
.#...##...
#..#.....#
....#...##
#.........
...##.#...
.##....#.#
##...#....
#.####...#

Tile 1907:
#.#####.#.
##..#.##..
.....#.###
.#.#..##.#
.#.###...#
...####...
#..###....
##....#...
..#..#.#..
#...###.#.

Tile 3373:
#.#####...
..###.....
.##..#..##
....#...##
......#..#
..##.....#
....##.###
.#.#.....#
#..##.#...
.......###

Tile 3253:
##..###..#
##......##
##.......#
##.#......
##.......#
#........#
..##...###
#..###..#.
#.....###.
..#.##..##

Tile 2767:
......####
..#...####
..###.....
#...##...#
...#..#..#
.####....#
#...##...#
...##.####
#...##.#..
..#.....##

Tile 3761:
....######
.......###
.#.##.....
.......#..
..........
.#.#.....#
#........#
##.##...##
#......#.#
#.##.#..#.

Tile 1031:
...#.###.#
.#......#.
##...#..#.
#...#.....
#.....#.##
#.#...#..#
##..##.#..
.##.#..#..
..#.....##
.......#.#

Tile 1181:
####..#.##
.....###.#
..##..#...
...#.#..##
...#....#.
#.......##
...#....##
#..##.#..#
###...#...
#.##..#.#.

Tile 2887:
.####..#.#
.......#.#
#.#..#.#.#
#.##.....#
...#.#...#
###..#....
#...#.....
.....#....
###..#..#.
...###..#.

Tile 1381:
#...#..#.#
..........
....#...#.
..#..##...
........##
#.#.......
##........
.#..##...#
#..#......
########..

Tile 2621:
...#.####.
#...#..#..
#.#.....#.
..##.#..##
####..#...
.###.#..#.
##...##..#
#..#.#..#.
#..##.#...
#..###...#

Tile 1597:
##.##..###
.#...##...
..##....#.
.##.......
.##....#.#
#..#.#....
#........#
...#......
.#..###.##
...###.###

Tile 3533:
#.###.##.#
##.##...#.
##..#.#.##
.#.#..#.##
#.....#...
#..#...#.#
..###.#..#
.#....#.#.
#.........
#...#.#.##

Tile 1249:
.##.....##
#.....#.##
#..#..#.##
#.#.#...#.
......#...
.#...##...
##.#.....#
..###.....
#.......#.
#.######..

Tile 2843:
#.#.#.....
.......#.#
#.....###.
#.....##.#
...##.##.#
#...#.####
..#.#....#
.......#..
##.#..#.##
.##.##...#

Tile 1583:
..####.#..
##..#....#
###...##.#
#....#.##.
#..#......
##.......#
##.#...#..
....#.#..#
....#..#.#
....#.###.

Tile 3169:
...#.###..
.####.....
#.#.#..#..
#...#..#..
..........
##.#...#..
..........
#..#....##
#...#....#
..##.....#

Tile 1367:
......##..
.##.......
##.#..##.#
#...#.#...
#.###.#..#
#.#...#..#
.....#...#
.....#...#
#.#.#.#...
#....##...

Tile 2099:
..###.####
.####.###.
...#...#..
...##.##.#
#....##...
....#..#.#
...#...##.
#........#
#..#.#..#.
..##...##.

Tile 3613:
#.#.####..
###......#
.##..##...
##..##...#
#........#
.....#....
.......#..
.##.#.....
.##.#....#
####..#.##

Tile 3727:
..#.##.##.
#..#..##..
##........
#.......#.
.#.###.#..
.#....#...
#.........
#...#.#.#.
.#..#...##
.###.#.#..

Tile 3697:
##.###.#..
......####
#...#.#...
.###...#.#
##.#....#.
..##.#..#.
.##....#.#
#..#.....#
.#........
#######.#.

Tile 2503:
..#.#..#..
.......###
....#.....
####.#.##.
...##...#.
#..#..#..#
..........
.......#..
##..#..#.#
.#...#.###

Tile 2131:
##..#...##
#..#.#..##
..##...##.
.....#.#.#
...##....#
..##..#..#
..........
......#.#.
.#..#.#.##
#..##.##..

Tile 2129:
.#.#....#.
.......#.#
.....#..#.
.#......#.
###.#.#...
##.##.#...
...#.##...
......####
....#####.
#..#.##.#.

Tile 3643:
##..#.....
.##.##...#
#....#...#
#.###.....
####.#....
.#.....#..
##..###.#.
..#..#.#.#
#..##.#..#
#...##..##

Tile 1307:
#......#.#
##...#..#.
......##..
....#.#..#
#....##..#
......#.##
....#.#..#
#....#....
#..#..#...
#..#.#.#..

Tile 3989:
..###.#..#
###...#...
..#..#....
##....#.#.
##..#.#..#
.......##.
...#..##.#
#....#....
......#...
####.#..#.

Tile 2347:
.#########
##...#.#.#
..#.##...#
#..#..##..
#..#......
###......#
.##...#...
.####...#.
#...#.##..
.#.#####..

Tile 2801:
#.###.####
......#..#
...##.....
#.....#..#
...#......
........##
..###.####
#.#.#.##..
###...##..
##.##..##.

Tile 2543:
.....#..##
#....#.#..
....#.##.#
##.#..###.
.#.#.#..##
#.###.#..#
##..#..#.#
...###.#..
.....###.#
.##..#.#.#

Tile 2029:
##.##..###
.........#
#.#......#
#.#.#..#.#
.#.#.##.##
#.....####
...#.#....
#..##....#
....#.#...
###.#..###

Tile 1889:
#..#.#####
#.#.#...##
#.#...##.#
.....#...#
##...#.###
.....#....
#.#......#
#........#
...#......
.#..#####.

Tile 1787:
######.#.#
#.........
.#.#.#....
...#.....#
#..#.#..##
##.#...#..
...#.....#
##..#.#...
#.....##..
#...##.###

Tile 1619:
#.####....
#.#.#.##..
#...#....#
.#........
.......#..
#....#.#.#
.......#.#
.#.#..##.#
.##..##.#.
.#...##...

Tile 2617:
#..##.###.
.###...#..
..#.....#.
#....##...
#...#...##
.#.....#.#
#....#...#
...##..#.#
........##
...###...#

Tile 2879:
#.#.####..
.....#....
.###...#..
...#......
#.#.....#.
....#.....
.#.#.#.#.#
......#.#.
#.........
#.#..#####

Tile 1429:
..#.......
#.##.#.###
#......###
....#...#.
.....#.#.#
..#....###
#..#......
....#..#..
..#...#...
.#####.#..

Tile 2797:
##.#.#..#.
.....#.#.#
.......#.#
#...#.#...
.##.#.##.#
.##.....##
##......#.
##.#.#.#..
.....#...#
##.#######

Tile 3943:
..#.#.####
.#.......#
#..#.##.#.
.#..#....#
.#.##....#
#.#.#.#..#
......#...
#...##...#
#....##...
.##....#.#

Tile 3691:
##.#...###
.#.##...#.
##.##....#
.##.......
......#..#
...#..#..#
####.....#
#..####...
..#.....#.
...#.#....

Tile 3083:
#.###.....
#.#....#.#
#..#...###
...#..#..#
..#..#....
##...#.#.#
#.#.......
#.#..###..
##.#.....#
.#.##.....

Tile 1013:
#....#..##
#..#....##
#.#.#.#...
.....#.#.#
#........#
..#.....#.
.##..#.###
#..#..#.#.
........##
#.#.#.#.##

Tile 2089:
##.#.#..#.
#...#...##
#...#...##
..#....###
...#......
......##..
#......###
...#...###
.##...#.#.
.##...###.

Tile 2551:
###.######
####.#.###
.##.##...#
#..##.##.#
#.#.#.#..#
....#.#...
#.#....#..
##...#...#
#..#.#...#
..###..#.#

Tile 3607:
.#...##..#
####......
#..#.##..#
#..#......
###.....#.
#....###..
.........#
....##.#.#
......#..#
...#####..

Tile 1831:
##....##..
##.#......
#.#....#..
#.##..##..
...###....
##.#.#...#
#...#.#...
....#....#
##......##
.##.#..#.#

Tile 1697:
##..###.#.
#..#...###
#..#.....#
......#...
...#......
#........#
.#....####
#....#....
..##.###..
.#.##..#..

Tile 1621:
....##.##.
....#.#...
....#.#.#.
#.#...#..#
..#..#.#..
#........#
....##....
#.###.#..#
#....#..##
#..####...

Tile 1019:
.#.###..#.
###.......
#...#.#..#
..#.#.....
#.....###.
....#..#.#
.#.#.#.#..
#.........
#...#..#.#
..##..####

Tile 1471:
...#.####.
...##.##..
#.#.#..###
..##...###
##.###....
#....#...#
........##
#.##....#.
##.##..#.#
####.###..

Tile 3797:
..##...##.
#...#....#
###.###.##
##....##..
#.#.#...##
....##....
#.....#..#
#...#.#...
#.##.#..#.
..#.#.#.#.

Tile 1231:
##....#..#
#.........
#..#.#....
##...#....
#..#.#...#
###.##....
##.#....#.
....#.#..#
.#...#..##
#..#####.#

Tile 1667:
##.##.#..#
...#..#.##
..........
#.........
..#.##...#
....#.#.##
..#.##...#
##...###.#
###.......
#####..###

Tile 3719:
..##.##..#
...#......
#......#.#
#.#..###..
....#.#.#.
#.........
#........#
##.......#
#.....#..#
.##.#.####

Tile 1103:
.#..####.#
#....##.##
....###..#
#.........
#..#.....#
#......#..
.#..###.#.
.....#....
.......#.#
.##.##.###

Tile 2609:
..####..#.
#.....#..#
.#.####...
....#.....
#......#.#
.........#
.......#..
#.##.#....
..#.#.....
####.##..#

Tile 1117:
.#.#..##.#
#...#....#
#.#..####.
.#.......#
#...#.....
#...#....#
#..#.#.#..
.........#
.....#.##.
.#..####.#

Tile 3359:
##...#.#.#
..#.#....#
..#..##...
.......#..
.........#
#.#...###.
..#..#...#
#..#.....#
#..#.#..##
.###.#.#.#

Tile 2851:
.#....####
#...##...#
#..#.....#
#......#..
#.......#.
.###.#.#..
#....#..#.
###......#
#.##.#..#.
..#...#.##

Tile 2423:
.#..#.####
...#....#.
.....##...
.........#
........##
##.......#
........#.
#....#..##
#.#.......
#...##.##.

Tile 1579:
.##....#.#
##...#....
.#.##....#
..##.#.###
.#..#.#..#
....##..#.
..#..##..#
#.#..##...
.........#
.#.###.###

Tile 1361:
....#..##.
.###.#...#
...#.#....
#..#..#...
#.##......
#..#...#..
.##......#
.#...#.#.#
#........#
.#.#######

Tile 1171:
##.####..#
....#.#..#
#..#.....#
#.#......#
###...#..#
.##.#...##
##..##.#.#
#..#.#..##
#......#.#
....#.###.

Tile 3847:
##...#..#.
##.#.#..##
..........
#........#
..........
#........#
#..#.#....
.#.....#.#
....#.....
#.##.##.##

Tile 2557:
##..#.#...
.#.#..#...
####....#.
#...#.#...
...#.....#
..#..##..#
..###.#.##
...#..#..#
.....##...
##.####..#

Tile 2069:
..###....#
###..#.###
........##
#..#.#..#.
.........#
##.#.#.#.#
....#...#.
...#.#....
........#.
#.#.##..##

Tile 2179:
##...#.#.#
.##..#.##.
#..#..#...
..........
###.#....#
..........
#...#.....
#........#
....#...#.
##..####..

Tile 2659:
##..#...#.
#....##...
.....#.#.#
.......#..
##.#..#.#.
##........
#....###.#
#......#.#
.........#
#...#.##.#

Tile 1097:
.#..#.###.
.....#..##
....####..
.#.#.....#
......##.#
.#..####.#
.###....#.
..##.##...
###.##.#..
#..###...#

Tile 2143:
.##.....#.
......#..#
.#........
##......##
###....##.
#...#....#
..#..#..##
..##....#.
#.#.......
#..#####.#

Tile 3527:
##.#...#..
..#.###..#
.........#
.....##.##
.#.....###
#.#...#..#
.#.##.#.##
###.#..#..
.........#
#####.#.##

Tile 1063:
##.#.#.##.
##...#....
#.........
##..#.#..#
......##.#
##....#.#.
.....#..##
##.###...#
.....##.#.
#......#.#

Tile 2593:
#...####.#
##.####...
##.#.#..#.
#........#
#...#....#
.........#
.#..#..#.#
......#...
.#.......#
##..#####.

Tile 1753:
...#.#..#.
#.....#...
.#......##
.#....##.#
.#..#...##
..#..###..
###......#
#.#.#..###
###....##.
..#.##...#

Tile 2273:
#..##.###.
#.....#...
#...##....
..#....###
#.........
#####.#...
##..###..#
#...##....
#.#......#
##.##..##.

Tile 2731:
.#..#..##.
...#...#.#
...##.....
#..#..#..#
..........
.......###
...#..####
##.##....#
...#.#..#.
.##..#..##

Tile 2477:
..#.###.##
#..#.#....
.#.....#..
##..#.##..
...##....#
#.#.#...##
.##......#
..##..#..#
..#......#
#..#..#.##

Tile 2243:
.#..######
..#..####.
...##.#...
...##..#.#
..#.#...#.
#.#.#..#..
..#......#
#...#.#...
...##...##
##.#####.#

Tile 1451:
#.####.#.#
#........#
#.#......#
.#.......#
.........#
..........
#.........
.........#
#.........
..#..#..##

Tile 1289:
#....#...#
#.....#.#.
...#......
.#....###.
......#..#
##........
###.......
.#...#..#.
..#..#.#.#
#.#...###.

Tile 2927:
..##.#..#.
##.#.#..#.
..########
#....##...
...#.##...
#.........
#.....#.#.
#.##..#...
....#.#...
#.##.#..##

Tile 3433:
#.##...###
#.#..#..#.
.#.....#.#
#..#......
.....##...
#.##.....#
#..#....##
.....#..##
##..#.....
..###..###

Tile 1913:
#.##..#.##
#..####.#.
#.##..##..
....#..#..
.#.....#..
.#..#....#
#..####.##
..#..#...#
#.......##
.#.#####..

Tile 1567:
####...##.
#..#.....#
##..#...#.
##.#.#.#.#
..#.#.#...
##.#..#..#
.##...##.#
....#...#.
..###..#..
...#..####

Tile 2707:
.#.#.#..##
.#....#..#
#..#.#...#
......#.#.
.#..##...#
.#.##.....
#.#.....##
#.....#..#
#..#..##.#
#..#...#.#

Tile 3803:
#..#.#...#
.#........
##....###.
#.##....##
..#....#..
##.#..#.#.
....#..#.#
.#.......#
#..#...#.#
##.....###

Tile 2837:
.#..#####.
#.......##
###..#.###
#...#....#
..#..#.#..
....#....#
.#..##.##.
...#.....#
#...#..#.#
.##...##.#

Tile 2473:
#.##..##..
....#..#.#
.........#
#.....##..
....#.#..#
...#..#...
....#...##
.....###..
..#......#
##..###..#

Tile 3121:
...#.##..#
#........#
.##..##...
#.....#..#
......##.#
......#.#.
......#..#
........##
#...#....#
.##.#..#..

Tile 1637:
#....#....
....#..#..
#......#.#
#..#..#..#
#.......##
#..##.....
#....#...#
.#........
.##..##.##
.##...###.

Tile 1091:
..###....#
###...#...
..#......#
##........
.........#
.....##..#
###.##....
#..#....#.
.#.#.#....
.....#####

Tile 1453:
###...#.##
.##..#.#.#
.#.......#
.##...#..#
#..#...#..
.##...#..#
#....##..#
...##..#..
....#.....
..##....##

Tile 2861:
##....####
.....#...#
.##.#...#.
.....##..#
##..##..#.
#...#.##.#
##....##.#
#..#.#....
#####.#...
##.....#..

Tile 2677:
##..#..#.#
#....#....
.#..#....#
.###....#.
.#........
.....#.#.#
#..#.....#
#.##.##.##
..#.##..##
#..#..#...

Tile 3413:
..###.#..#
...##.....
...#..####
........#.
##.....#.#
......#..#
#...#..#.#
#.......#.
##..#.#.#.
#...#####.

Tile 1193:
#.#.#.#.#.
.#....##..
#.........
.......#.#
..#.#.#.##
...#...##.
##........
#..#....##
.........#
####...###

Tile 1949:
##..#..##.
.....#..#.
...##...##
####....##
.#..#.....
#........#
.##.#....#
...##.....
...#..####
###...####

Tile 3863:
#.#..#.###
#.#.....##
##....##.#
..###.###.
#.##....#.
...#.#.###
...#..#.##
..#......#
##...#...#
#..#.#.##.

Tile 1439:
#....#.###
.....#.#.#
.#.#...##.
.##.#....#
#....#....
....#.#..#
.....#...#
#...###...
.#..##..##
##..#.####

Tile 1129:
..########
..#.#.#...
....##....
.......#..
........##
..........
#..#.#.#..
##........
.........#
..##..####

Tile 2953:
.#.###.###
........#.
....#.#.##
..#.#....#
......#.##
...#....#.
###...#..#
....#.....
#.........
#...#...#.

Tile 3371:
##..#..###
##.....###
##....#..#
#.....#...
###.#..###
.#.#..#.##
##........
##.......#
#.##..#...
..##.####.

Tile 1399:
#.....#.#.
#......#..
.#..#....#
####..#..#
#..#.....#
##..#..#..
#.....#.##
.#..##..#.
.#..#....#
..#.#.###.

Tile 3911:
.#.....#.#
..#.....##
#...##....
#.........
..#.#...##
..#...#..#
.#.#....##
#..##....#
.#..##..##
#.#..#..#.

Tile 1699:
.####.####
..#..#....
#.....#.#.
...#..##..
##.#..#...
##..##....
#..#..#.#.
.###.#....
.....##.##
.#..#...##

Tile 3701:
.#.#.#.#..
..#.####.#
..##...###
......##.#
....#....#
.....#..##
...#.#.###
#..####.#.
...##....#
..###..###

Tile 1811:
.##.##.#..
....#....#
#......#.#
.....#..#.
#..#.#...#
.#.#.##..#
......#...
..##..#...
.....#...#
#####.#..#

Tile 2539:
...######.
#...#.###.
.#..#.....
.....#...#
.#...#...#
#.###..###
.######...
#...#.#.##
#.....##.#
...#..#.##

Tile 3517:
.#...#.##.
#....#..#.
#......#.#
##.##.....
...###..#.
#..#.#...#
#...#.#...
###.......
.#.......#
######.#..

Tile 2113:
#..###..#.
...#....##
##.......#
#....##..#
..#.#..###
#.#......#
#.....#..#
.#..#.#..#
...###....
##.#..#...

Tile 1009:
...##..###
..#.......
.#.....##.
##....#..#
##........
#....#..#.
.........#
#.#..#..##
....#.#..#
..#.##.#.#

Tile 1559:
###....#.#
.###...#.#
#.#.....#.
.#..#.#..#
....##.#.#
#.#..##.##
..#..##...
#.###.....
.#.#.##..#
##....####

Tile 2971:
...###...#
......##..
...#....##
.#.#.##..#
###.#.##.#
#..#...##.
..#.#...##
.....#.##.
.#........
##....#..#

Tile 1327:
.#..##...#
#.##.....#
.#.....#.#
...###...#
#....#....
.#.......#
.#.......#
#.........
.....#...#
..#######.

Tile 3217:
..###..#..
.#..#....#
......####
###....#.#
###...#...
.#...#....
........##
#.##.#.##.
#......#..
...#...#.#

Tile 1861:
####.####.
.#.....#..
###...#...
.##.#..##.
...#..#...
...#..#...
##..#...##
#...##..#.
#...#.....
##........

Tile 3593:
.##.##.###
....#...##
.#..#....#
#..##.##..
#..#...###
...#..##..
....#...##
###......#
..........
..#......#

Tile 3041:
###..###.#
....####..
....#..##.
#.#.....##
#..#...#..
#.......##
.#...#####
##.#...#.#
..#.###.#.
.####...##

Tile 2647:
#.###.####
#...#...##
.##....##.
#.#.#.....
#..#..##.#
##....#..#
..#....###
.....###..
........##
#.#.#....#

Tile 2437:
.##...#...
#.#...##.#
.#....##.#
#####...#.
#...##.#..
..#....#..
#.......#.
..#..#.#..
#.##...#.#
#.##..#..#

Tile 3709:
#######.##
###.#..##.
#.#...#..#
.#....####
##...#..#.
....#..#..
##........
#..##..#.#
.....#...#
..#.##...#

Tile 2039:
##.###..#.
...#.....#
..#...##.#
##.#.#...#
#....#..#.
#.####...#
...##...##
##...##..#
#.....##.#
####.#.#.#

Tile 3011:
...##.#...
.........#
###.####.#
#.......#.
.#.....#.#
..######..
#.....#..#
#........#
.##.##....
#.....###.

Tile 1151:
.##.#..#.#
#.##.....#
...#..#...
....##..##
###....###
##..#..#.#
#...#....#
..#....#..
...#.....#
#.#..#.#.."""

## La première étoile

La première étoile est proche, il me reste à
- lire les fragments ci-dessus
- les transformer en objet Tile
- soumettre cette liste au résolveur
- croiser les doigts
- soumettre la réponse

Ce qui est demandé sur cette première étoile est simplement de prendre les identifiants des 4 coins du puzzle et de les multiplier ensemble.

C'est parti :

In [6]:
// construction des Tile
val parsedTiles =  tiles.split("\n\n").map { tile ->
            tile.lines().let { lines ->
                TileBuilder(
                    lines.first().substringAfter("Tile ").substringBefore(":").toLong(), 
                    lines.drop(1).joinToString("\n")
                )
            }
        }

          
// soumission au puzzle
PuzzleMaker(parsedTiles).run {
    
    // assemblage du puzzle
    fill()
    
    // multiplication des identifiants des 4 coins :
    puzzle.first().first()!!.id * 
    puzzle.first().last()!!.id * 
    puzzle.last().first()!!.id * 
    puzzle.last().last()!!.id
    
}

23386616781851

### Le puzzle terminé

La réponse était donc **23386616781851**

Et voici le rendu du puzzle :

![alt text](day20puzzle.png "Le puzzle")

## Les monstres

J'ai assemblé mon puzzle, mais je vois pas de monstres là dedans !

Ah la la je m'aperçois que les bords de chaque fragments n'étaient là que pour l'assemblage du puzzle, il faut ensuite les enlever pour obtenir la vraie image ^^

### Opération découpage des bords

In [7]:
// nettoyage des bords d'un fragment
fun Tile.removeBorders() = display.let {
       it.lines().toMutableList().drop(1).dropLast(1).map {
           it.toList().drop(1).dropLast(1).joinToString("")
       }.joinToString("\n")
   }

fun SquarePuzzleMaker.drawFinalImage() = puzzle.map { it.map { it!!.removeBorders() } }.let {
       it.map { tiles -> (tiles.first().lines().indices).map { indice ->
           tiles.joinToString("") { it.lines()[indice] }
       }.joinToString("\n")
       }.joinToString("\n")
   }

val finalImage = PuzzleMaker(parsedTiles).run {
    
    // assemblage du puzzle
    fill()
    
    drawFinalImage()
    
}
finalImage

###......###.#..#......#.....#.......#..#..#.#.##.....#..#.....##.#...##....#....#......##...#..
..#.##..#.#............##.......#...#..#.......#..####.....##............#......##.#.##......#..
..#...........#...#.....#......#.#.#.#.#...#..#......###..............##..#...#.#.........#.....
##.....#.........##.....##....##..#.#.#......#....#..#...#.#...#..#.#..##.....#....#...#.....#..
....###.##.............#...#......#..#.##.....#...#......#...#........##..#.#.....#...##........
............##.#...#.....#..#..#.##...##.......#......###.#....#.#.....#..#..#...#....#####....#
...##.#.#....#..###......##....##...#.....#..###....##.#..##......#......#.....####........#....
.....#...#...#.###..#....#.......#..###...##....#...##.##..##..#......#.##....#.....#..#.###.##.
.##...##.........##.....#...#..#.......###...#..##..#.#.......#....##.#.#.#..#.........##...#.#.
##.#........#........#.....#.#......#..#.#..#...#.......##.......#..#...###....##..####.#.#....#
..........#...##.....#.##.#...

Ben je vois toujours pas de monstres moi ... je commence à comprendre pourquoi le doute plane toujours près d'Inverness ...

Bon, mes dernières informations m'indiquent que le monstre a toujours cette forme :

```
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
```

Alors ok, je vais le chercher dans ce brouillard ! 
En fait ce qu'on me demande, c'est de compter le nombre de # de l'image qui n'appartiennent pas au monstre.

Alors ok, comptons !
Je vais déjà compter le nombre de monstres ...

In [8]:
fun String.findMonsters(): Int = 
    
    // construction des coordonnées des points de départ potentiels des monstres
    lines().flatMapIndexed {
        y, line -> line.toList().mapIndexed { 
            // le monstre fait une longueur de 20 et le haut de sa tete se situe à l'indice 18
            // => aucun # avant l'insdice 18 ne peut etre le départ d'un monstre
            // le monstre fait une hauteur de 3 
            // => les deux dernières lignes ne peuvent etre le départ d'un monstre
            x, cell -> if (x >= 18 && y <= lines().size - 2 && cell == '#') x to y else null 
        } 
    }
    .filterNotNull()
    .count { (x, y) ->
        // pour chaque coordonnée, je vérifie que tous les éléments du corps sont présents
        lines()[y+1].toList().get(x-18) == '#'
        && lines()[y+1].toList().get(x-18 + 5) == '#'
        && lines()[y+1].toList().get(x-18 + 6) == '#'
        && lines()[y+1].toList().get(x-18 + 11) == '#'
        && lines()[y+1].toList().get(x-18 + 12) == '#'
        && lines()[y+1].toList().get(x-18 + 17) == '#'
        && lines()[y+1].toList().get(x-18 + 18) == '#'
        && lines()[y+1].toList().get(x-18 + 19) == '#'
        && lines()[y+2].toList().get(x-18 + 1) == '#'
        && lines()[y+2].toList().get(x-18 + 4) == '#'
        && lines()[y+2].toList().get(x-18 + 7) == '#'
        && lines()[y+2].toList().get(x-18 + 10) == '#'
        && lines()[y+2].toList().get(x-18 + 13) == '#'
        && lines()[y+2].toList().get(x-18 + 16) == '#'
               }

   
finalImage.findMonsters()

0

0 ??? Eh oui, je regarde sûrement mon image dans le mauvais sens ^^ 

Voyons si en la tournant je détecte des monstres...


## La deuxième étoile

In [9]:
// je vais la tourner dans tous les sens et garder celui pour lequel j'aurai trouvé le plus de monstres
val monsters = finalImage
    .rotations()
    .fold(0) { monsters, image -> image.findMonsters().let { if (it > monsters) it else monsters } }
monsters

19

Bingo, j'en ai trouvé 19, tout ça est un peu lent et mériterait de réécrire quelques trucs, mais quoi, après tout je rêve de vacances, pas étonnant que je ne sois plus en mesure de trouver l'algo parfait du premier coup mouahaha !

Bon allez, j'y suis presque, la question n'était pas de savoir combien de monstres se trouvaient dans l'eau (faites moi penser à transmettre mes informations à Inverness), mais bien de savoir combien de # n'appartenaient pas à des monstres :

In [10]:
// un monstre comprend 15 caractères, je les soustrais au total
finalImage.count { it == '#' } - monsters * 15

2376

Le résultat de la deuxième étoile était donc **2376** !!!

Bonne nuit, je vais me coucher ^^



Hein ?
Quoi ? 

Vous ne les avez pas vus ?
Et vous voulez les voir ?

Bon, ok, mais rapido crahouète alors !

In [11]:
fun String.findMonstersPoints(): List<Pair<Int, Int>> = 
    
    // construction des coordonnées des points de départ potentiels des monstres
    lines().flatMapIndexed {
        y, line -> line.toList().mapIndexed { 
            // le monstre fait une longueur de 20 et le haut de sa tete se situe à l'indice 18
            // => aucun # avant l'insdice 18 ne peut etre le départ d'un monstre
            // le monstre fait une hauteur de 3 
            // => les deux dernières lignes ne peuvent etre le départ d'un monstre
            x, cell -> if (x >= 18 && y <= lines().size - 2 && cell == '#') x to y else null 
        } 
    }
    .filterNotNull()
    .flatMap { (x, y) ->
        // pour chaque coordonnée, je vérifie que tous les éléments du corps sont présents
        if (lines()[y+1].toList().get(x-18) == '#'
        && lines()[y+1].toList().get(x-18 + 5) == '#'
        && lines()[y+1].toList().get(x-18 + 6) == '#'
        && lines()[y+1].toList().get(x-18 + 11) == '#'
        && lines()[y+1].toList().get(x-18 + 12) == '#'
        && lines()[y+1].toList().get(x-18 + 17) == '#'
        && lines()[y+1].toList().get(x-18 + 18) == '#'
        && lines()[y+1].toList().get(x-18 + 19) == '#'
        && lines()[y+2].toList().get(x-18 + 1) == '#'
        && lines()[y+2].toList().get(x-18 + 4) == '#'
        && lines()[y+2].toList().get(x-18 + 7) == '#'
        && lines()[y+2].toList().get(x-18 + 10) == '#'
        && lines()[y+2].toList().get(x-18 + 13) == '#'
        && lines()[y+2].toList().get(x-18 + 16) == '#') 
            listOf(x to y, x-18 to y+1, x-18+5 to y+1, x-18+6 to y+1, x-18+11 to y+1, x-18+12 to y+1, x-18+17 to y+1, x-18+18 to y+1, x-18+19 to y+1, x-18+1 to y+2, x-18+4 to y+2, x-18+7 to y+2, x-18+10 to y+2, x-18+13 to y+2, x-18+16 to y+2) 
        else 
            listOf(null)
    }.filterNotNull()

val monstersCoordinates = finalImage
    .rotations()
    .fold(emptyList<Pair<Int, Int>>() to "") { monsters, image -> image.findMonstersPoints().let { if (it.size > monsters.first.size) it to image else monsters } }

monstersCoordinates.first.size / 15

19

In [16]:
monstersCoordinates.second.lines().mapIndexed {
    y, line -> line.toList().mapIndexed { x, c ->
        if (monstersCoordinates.first.contains(x to y)) "\u2588" else c
    }.joinToString("")
}.joinToString("\n")

#..#.....#.....#..#.......##.#.....#......#..#..#...#..#..#...##....#..#...#......#...####.#.##.
#..#....##.#..#..#.#........#...##...#..#........#......#.#.#.#.....#.#..#......#..#.........#..
###.....#..##.#.#.#.......###.#...#█#...##...#.#..#...................#..#.##...#...#.#.....##..
......#..#...#...█#.#.██.#..██#...███.#.....#.#........#..#.#.#....###.........#...#....##.#..#.
.#..#.#......#.##.█..█.#█..█..█#.█..#......##..........##.##.#.........##..###.....#..##.#......
.#..#..#...#....#...##...##.#..#.....#..#..#..#..##......#......#........###....#..#......#..#..
....#.#.#...#..#.#..##.####..###.#..#.#...#......#.#........#.#...#...##......#............#...#
...#....#......#..###..#..##.....#....#....##..#...##.....#...#.....#........#...#.##..█.##.....
.#..#.#....##.##...#.#........#.............#...##.......###.#.#..#.#█.#..██....██#..#███...#...
#...#..#.............#..........#.....#..##..##.....#.......####......█#.█..█.#█..█.#█#.......##
##........#.#.#..##.#...#.##..